<a href="https://colab.research.google.com/github/RvoRvo/learn-pyspark/blob/main/notebooks/load_csv_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("LearnPySpark")
    .getOrCreate()
)


In [5]:
import urllib.request

url = "https://raw.githubusercontent.com/RvoRvo/learn-pyspark/main/notebooks/data/customers.csv"
local_path = "/content/customers.csv"

urllib.request.urlretrieve(url, local_path)
print("Downloaded to:", local_path)


Downloaded to: /content/customers.csv


In [6]:
df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/content/customers.csv")
)

df.show()


+-----------+----------+---------+--------------------+---------+-----------+
|customer_id|first_name|last_name|               email|  country|signup_date|
+-----------+----------+---------+--------------------+---------+-----------+
|          1|      John|    Smith|john.smith@email.com|   Canada| 2024-01-15|
|          2|     Maria| Gonzalez|maria.gonzalez@em...|      USA| 2024-01-18|
|          3|     David|    Brown|david.brown@email...|       UK| 2024-02-02|
|          4|    Sophie|   Martin|sophie.martin@ema...|   France| 2024-02-10|
|          5|      Alex|     Chen| alex.chen@email.com|   Canada| 2024-02-21|
|          6|      Emma|   Wilson|emma.wilson@email...|Australia| 2024-03-05|
|          7|    Daniel|     Levi|daniel.levi@email...|   Israel| 2024-03-12|
|          8|     Laura|    Rossi|laura.rossi@email...|    Italy| 2024-03-20|
+-----------+----------+---------+--------------------+---------+-----------+



In [8]:
import urllib.request

url = "https://raw.githubusercontent.com/RvoRvo/learn-pyspark/main/notebooks/data/orders.csv"
local_path = "/content/orders.csv"

urllib.request.urlretrieve(url, local_path)
print("Downloaded to:", local_path)

Downloaded to: /content/orders.csv


In [9]:
customers_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/content/customers.csv")
)

orders_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/content/orders.csv")
)


In [11]:
joined_df = (
    orders_df
    .join(customers_df, on="customer_id", how="inner")
)

joined_df.show()


+-----------+--------+----------+------+----------+---------+--------------------+-------+-----------+
|customer_id|order_id|order_date|amount|first_name|last_name|               email|country|signup_date|
+-----------+--------+----------+------+----------+---------+--------------------+-------+-----------+
|          1|    1003|2024-02-10| 15.75|      John|    Smith|john.smith@email.com| Canada| 2024-01-15|
|          1|    1001|2024-02-01|  25.5|      John|    Smith|john.smith@email.com| Canada| 2024-01-15|
|          2|    1002|2024-02-03|  80.0|     Maria| Gonzalez|maria.gonzalez@em...|    USA| 2024-01-18|
|          3|    1005|2024-03-01|  42.0|     David|    Brown|david.brown@email...|     UK| 2024-02-02|
|          5|    1004|2024-02-18| 120.0|      Alex|     Chen| alex.chen@email.com| Canada| 2024-02-21|
|          8|    1006|2024-03-05|  60.0|     Laura|    Rossi|laura.rossi@email...|  Italy| 2024-03-20|
+-----------+--------+----------+------+----------+---------+------------

In [12]:
joined_df.select(
    "order_id",
    "order_date",
    "customer_id",
    "first_name",
    "last_name",
    "country",
    "amount"
).show()


+--------+----------+-----------+----------+---------+-------+------+
|order_id|order_date|customer_id|first_name|last_name|country|amount|
+--------+----------+-----------+----------+---------+-------+------+
|    1003|2024-02-10|          1|      John|    Smith| Canada| 15.75|
|    1001|2024-02-01|          1|      John|    Smith| Canada|  25.5|
|    1002|2024-02-03|          2|     Maria| Gonzalez|    USA|  80.0|
|    1005|2024-03-01|          3|     David|    Brown|     UK|  42.0|
|    1004|2024-02-18|          5|      Alex|     Chen| Canada| 120.0|
|    1006|2024-03-05|          8|     Laura|    Rossi|  Italy|  60.0|
+--------+----------+-----------+----------+---------+-------+------+



In [14]:
orders_df.join(customers_df, "customer_id", "left").show()
orders_df.join(customers_df, "customer_id", "right").show()
orders_df.join(customers_df, "customer_id", "full").show()



+-----------+--------+----------+------+----------+---------+--------------------+-------+-----------+
|customer_id|order_id|order_date|amount|first_name|last_name|               email|country|signup_date|
+-----------+--------+----------+------+----------+---------+--------------------+-------+-----------+
|          1|    1001|2024-02-01|  25.5|      John|    Smith|john.smith@email.com| Canada| 2024-01-15|
|          2|    1002|2024-02-03|  80.0|     Maria| Gonzalez|maria.gonzalez@em...|    USA| 2024-01-18|
|          1|    1003|2024-02-10| 15.75|      John|    Smith|john.smith@email.com| Canada| 2024-01-15|
|          5|    1004|2024-02-18| 120.0|      Alex|     Chen| alex.chen@email.com| Canada| 2024-02-21|
|          3|    1005|2024-03-01|  42.0|     David|    Brown|david.brown@email...|     UK| 2024-02-02|
|          8|    1006|2024-03-05|  60.0|     Laura|    Rossi|laura.rossi@email...|  Italy| 2024-03-20|
+-----------+--------+----------+------+----------+---------+------------